In [ ]:
import pandas as pd 
import numpy as np 
import anndata as an 
import sys, os 
sys.path.append('../src')

import sys
import os
import pathlib 
from typing import *

import torch
import numpy as np 
import pandas as pd 
import anndata as an
from functools import cached_property

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import LabelEncoder 

from models.lib.data import *
from models.lib.lightning_train import *
from models.lib.neural import *
from data.downloaders.external_download import *

from torchmetrics.functional import *

In [2]:
module = DataModule(
    datafiles=['../data/dental/human_dental_T.h5ad'],
    labelfiles=['../data/dental/labels_human_dental.tsv'],
    class_label='cell_type',
    sep='\t',
    batch_size=16,
    num_workers=0,
)

In [3]:
labels = pd.read_csv('../data/dental/labels_human_dental.tsv', sep='\t')
labels.shape

(41673, 6)

In [4]:
labels['cell_type'].nunique()

9

In [3]:
early_stop_callback = EarlyStopping(
    monitor="weighted_val_accuracy", 
    min_delta=0.00, 
    patience=3, 
    verbose=False, 
    mode="max"
)

model = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
    weighted_metrics=True,
)

wandb_logger = WandbLogger(
    project=f"tabnet-classifer-sweep",
    name='WEIGHTED Dental model local'
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop_callback],
    max_epochs=1,
)

trainer.fit(model, datamodule=module)

Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.4 M 
-----------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.770     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: _standard_collate() got an unexpected keyword argument 'noise_func'